# CAR GAME

In [1]:
import cv2
import cvzone
import random
from cvzone.HandTrackingModule import HandDetector
from cvzone.FaceMeshModule import FaceMeshDetector
import numpy as np
import pygame
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize pygame mixer
pygame.mixer.init()
point_s = pygame.mixer.Sound("point.wav")
gameover_s = pygame.mixer.Sound("gameOver.wav")

In [3]:
# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

detector = HandDetector(detectionCon=0.8, maxHands=1)
faceDetector = FaceMeshDetector(maxFaces=1)

I0000 00:00:1753121032.630607 45041718 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
I0000 00:00:1753121032.653561 45041718 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1753121032.666893 45042570 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753121032.667474 45042559 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753121032.674393 45042570 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753121032.697945 45042559 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
# Load images (with error handling)
try:
    car_img = cv2.imread('car.png', cv2.IMREAD_UNCHANGED)
    car_img = cv2.resize(car_img, (100, 100))
except:
    car_img = None

try:
    obstacle_img = cv2.imread('obstacle.png', cv2.IMREAD_UNCHANGED)
    obstacle_img = cv2.resize(obstacle_img, (80, 80))
except:
    obstacle_img = None

try:
    coin_img = cv2.imread('coin.png', cv2.IMREAD_UNCHANGED)
    coin_img = cv2.resize(coin_img, (40, 40))
except:
    coin_img = None

try:
    mask_img = cv2.imread('spiderman.png', cv2.IMREAD_UNCHANGED)
except:
    mask_img = None

In [5]:
# Game variables
player_x = 110  # Center lane x
lanes = [10, 110, 210]  # Left, center, right lane x positions
player_y = 350
player_width = 100
player_height = 100

objects = []  # [type, x, y, width, height]

score = 0
gameOver = False
game_started = False

# Timer for object spawning
last_spawn_time = time.time()

In [ ]:
while True:
    success, img = cap.read()
    if not success:
        break
    img = cv2.flip(img, 1)

    # Detect face mesh and overlay mask
    img, faces = faceDetector.findFaceMesh(img, draw=False)
    if faces and mask_img is not None:
        face = faces[0]
        
        # Get key landmarks for alignment
        left_eye = face[145]
        right_eye = face[374]
        nose = face[1]
        
        # Calculate mask width based on eye distance
        eye_width = int(((right_eye[0] - left_eye[0]) ** 2 + (right_eye[1] - left_eye[1]) ** 2) ** 0.5 * 2.5)
        mask_height = int(eye_width * mask_img.shape[0] / mask_img.shape[1])
        
        # Position: center mask at nose tip
        x1 = int(nose[0] - eye_width // 2)
        y1 = int(nose[1] - mask_height // 2)
        
        # Resize mask
        resized_mask = cv2.resize(mask_img, (eye_width, mask_height))
        
        # Overlay mask
        img = cvzone.overlayPNG(img, resized_mask, [x1, y1])

    # Split screen canvas
    canvas = np.zeros((480, 640, 3), dtype=np.uint8)

    # Left half: webcam feed with hand tracking and mask
    hands, img = detector.findHands(img, flipType=False)
    
    # Create hand mask (for visualization)
    hand_mask = np.zeros_like(img)
    if hands:
        hand = hands[0]
        for lm in hand["lmList"]:
            cv2.circle(hand_mask, (lm[0], lm[1]), 5, (255, 0, 255), cv2.FILLED)
        
        if hasattr(detector, 'handConnections'):
            connections = detector.handConnections
        else:
            connections = [[0,1],[1,2],[2,3],[3,4],
                           [0,5],[5,6],[6,7],[7,8],
                           [5,9],[9,10],[10,11],[11,12],
                           [9,13],[13,14],[14,15],[15,16],
                           [13,17],[17,18],[18,19],[19,20],[0,17]]
        
        for connection in connections:
            x1, y1 = hand["lmList"][connection[0]][0], hand["lmList"][connection[0]][1]
            x2, y2 = hand["lmList"][connection[1]][0], hand["lmList"][connection[1]][1]
            cv2.line(hand_mask, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    img_with_mask = cv2.addWeighted(img, 0.7, hand_mask, 0.3, 0)
    img_resized = cv2.resize(img_with_mask, (320, 480))
    canvas[:, :320] = img_resized

    # Right half: game area
    game_area = np.zeros((480, 320, 3), dtype=np.uint8)
    game_area[:, :] = (50, 50, 50)  # Dark grey background

    for y in range(0, 480, 40):
        cv2.rectangle(game_area, (50, y), (70, y+20), (255, 255, 255), -1)
        cv2.rectangle(game_area, (170, y), (190, y+20), (255, 255, 255), -1)

    if not game_started:
        cv2.putText(game_area, "TEMPLE RUN", (40, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 3)
        cv2.putText(game_area, "Press SPACE to Start", (20, 240), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        canvas[:, 320:] = game_area
        cv2.imshow("Temple Run Split Screen", canvas)
        key = cv2.waitKey(1)
        if key == ord(' '):
            game_started = True
        elif key == ord('q'):
            break
        continue

    if gameOver:
        cv2.putText(game_area, "GAME OVER", (30, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        cv2.putText(game_area, f"Score: {score}", (80, 220), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.putText(game_area, "Press SPACE to Restart", (10, 260), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        canvas[:, 320:] = game_area
        cv2.imshow("Temple Run Split Screen", canvas)
        key = cv2.waitKey(1)
        if key == ord(' '):
            gameOver = False
            score = 0
            objects.clear()
            game_started = True
        elif key == ord('q'):
            break
        continue

    if hands and not gameOver:
        xHand = hands[0]['lmList'][8][0]
        if xHand < 213:
            player_x = lanes[0]
        elif xHand < 426:
            player_x = lanes[1]
        else:
            player_x = lanes[2]

    if car_img is not None:
        game_area = cvzone.overlayPNG(game_area, car_img, [player_x, player_y])
    else:
        cv2.rectangle(game_area, (player_x, player_y), (player_x + player_width, player_y + player_height), (255, 0, 0), -1)
        cv2.rectangle(game_area, (player_x, player_y), (player_x + player_width, player_y + player_height), (0, 0, 0), 3)

    current_time = time.time()
    if current_time - last_spawn_time > 1 and not gameOver:
        obj_type = random.choice(['obs', 'coin'])
        lane_x = random.choice(lanes)
        if obj_type == 'obs':
            objects.append(['obs', lane_x, -80, 80, 80])
        else:
            objects.append(['coin', lane_x + 30, -60, 40, 40])
        last_spawn_time = current_time

    for obj in objects[:]:
        if not gameOver:
            obj[2] += 10

        if obj[0] == 'obs':
            if obstacle_img is not None:
                game_area = cvzone.overlayPNG(game_area, obstacle_img, [obj[1], obj[2]])
            else:
                cv2.circle(game_area, (obj[1] + 40, obj[2] + 40), 40, (0, 0, 255), -1)
                cv2.circle(game_area, (obj[1] + 40, obj[2] + 40), 40, (0, 0, 0), 3)
            
            if not gameOver:
                player_rect = [player_x, player_y, player_width, player_height]
                obstacle_rect = [obj[1], obj[2], obj[3], obj[4]]
                
                if (player_rect[0] < obstacle_rect[0] + obstacle_rect[2] and
                    player_rect[0] + player_rect[2] > obstacle_rect[0] and
                    player_rect[1] < obstacle_rect[1] + obstacle_rect[3] and
                    player_rect[1] + player_rect[3] > obstacle_rect[1]):
                    gameover_s.play()
                    gameOver = True
        else:
            if coin_img is not None:
                game_area = cvzone.overlayPNG(game_area, coin_img, [obj[1], obj[2]])
            else:
                cv2.ellipse(game_area, (obj[1] + 20, obj[2] + 20), (20, 20), 0, 0, 360, (0, 255, 255), -1)
                cv2.ellipse(game_area, (obj[1] + 20, obj[2] + 20), (20, 20), 0, 0, 360, (0, 0, 0), 2)
            
            if not gameOver:
                player_rect = [player_x, player_y, player_width, player_height]
                coin_rect = [obj[1], obj[2], obj[3], obj[4]]
                
                if (player_rect[0] < coin_rect[0] + coin_rect[2] and
                    player_rect[0] + player_rect[2] > coin_rect[0] and
                    player_rect[1] < coin_rect[1] + coin_rect[3] and
                    player_rect[1] + player_rect[3] > coin_rect[1]):
                    score += 1
                    point_s.play()
                    objects.remove(obj)

        if obj[2] > 480:
            objects.remove(obj)

    cv2.putText(game_area, f"Score: {score}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    canvas[:, 320:] = game_area

    cv2.imshow("Temple Run Split Screen", canvas)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


/Users/ankitkumar/Library/Python/3.9/lib/python/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
2025-07-21 23:34:01.957 Python[92544:45041718] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


KeyboardInterrupt: 

: 